# Zillow Logerror Estimation Model

## Project Goals
1. create a model that can predict error in Zestimated values

2. from that model, identify key features driving logerror

3. clearly communicate finding to classmates

## Project Requirements
1. utilize clustering algorithms at some point in the pipeline

2. utilize statistical testing to identify key features

3. provide helpful visulatizations explaining exploration process

4. use scaling methods on data and document why they we're used

5. impute missing values and document

6. encode cateorical data

7. feature engineering and document rational behind it


### 0. Imports

In [3]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#python libraries used
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#project specific functions
import acquire
import prepare
import explore
import modeling
import cluster
from env import host,password,user



### I. Acquisition

Get data on homes sold in 2017 with recorded Latitude and Longitude

In [25]:
#mysql query
query = '''
    SELECT prop.*, pred.logerror, pred.transactiondate
    FROM predictions_2017 AS pred
    LEFT JOIN properties_2017 AS prop  USING(parcelid)
    WHERE (bedroomcnt > 0 AND bathroomcnt > 0 AND calculatedfinishedsquarefeet > 500 AND latitude IS NOT NULL AND longitude IS NOT NULL) 
    AND (unitcnt = 1 OR unitcnt IS NULL)
    ;
    '''
db_name = 'zillow'
#get data from database
zillow = pd.read_sql(query, f"mysql+pymysql://{user}:{password}@{host}/{db_name}")
print(f'Data frame shape:\nrows: {zillow.shape[0]}\ncolumns: {zillow.shape[1]}')

Data frame shape:
rows: 73596
columns: 61


## II. Preparation

Standardize all null values to *np.nan*

In [26]:
zillow.fillna(value=pd.np.nan, inplace=True)

Add some features I think will be significant

In [27]:
zillow['has_basement'] = zillow.basementsqft > 0
zillow['has_fireplace'] = zillow.fireplacecnt > 0
zillow['has_deck'] = ~zillow.decktypeid.isna()
zillow['has_garage'] = zillow.garagetotalsqft > 0
zillow['has_pool_or_spa'] = (zillow.hashottuborspa == 1) | (zillow.poolcnt> 0)
zillow['has_yardbuilding'] = (zillow.yardbuildingsqft17) > 0 | (zillow.yardbuildingsqft26 > 0)
zillow['multistory'] = zillow.numberofstories > 1

Drop columns that are missing over 25% of their data and rows that are missing over 60% 

In [28]:
zillow = prepare.handle_missing_values(zillow, prop_required_column= .25, prop_required_row=.60)
print(f'Data frame now has:\nrows: {zillow.shape[0]}\ncolumns: {zillow.shape[1]}')

Data frame now has:
rows: 73596
columns: 43


Now we can clean up column names

In [30]:
zillow.rename(columns = {
    'parcelid': 'parcel_id',
    'airconditioningtypeid': 'ac_type_id',
    'bathroomcnt': 'bathroom_cnt',
    'bedroomcnt': 'bedroom_cnt',
    'buildingqualitytypeid': 'building_quality_type',
    'calculatedbathnbr': 'sum_bath_and_bed',
    'calculatedfinishedsquarefeet': 'square_feet',
    'fips': 'fips_code',
    'fullbathcnt': 'full_bath_cnt',
    'garagecarcnt': 'garage_car_cnt',
    'garagetotalsqft': 'garage_sqr_ft',
    'heatingorsystemtypeid': 'heating_type_id',
    'lotsizesquarefeet': 'lot_sqr_ft',
    'propertycountylandusecode': 'property_land_use_code',
    'propertylandusetypeid': 'property_land_use_id',
    'propertyzoningdesc': 'property_zoning',
    'rawcensustractandblock': 'raw_census_block',
    'regionidcity': 'city_id',
    'regionidcounty': 'county_id',
    'regionidneighborhood': 'neighborhood_id',
    'regionidzip': 'zipcode_id',
    'roomcnt': 'room_cnt',
    'unitcnt': 'unit_cnt',
    'yearbuilt': 'year_built',
    'structuretaxvaluedollarcnt': 'building_value',
    'taxvaluedollarcnt': 'total_value',
    'assessmentyear': 'year_assessed',
    'landtaxvaluedollarcnt': 'land_value',
    'taxamount': 'tax_amount',
    'censustractandblock': 'census_block',
    'transactiondate': 'transaction_date'
    }, inplace= True)
zillow.head()

,id,parcel_id,ac_type_id,bathroom_cnt,bedroom_cnt,building_quality_type,sum_bath_and_bed,square_feet,finishedsquarefeet12,fips_code,...,census_block,logerror,transaction_date,has_basement,has_fireplace,has_deck,has_garage,has_pool_or_spa,has_yardbuilding,multistory
0,1727539,14297519,NaN,3.5,4.0,NaN,3.5,3100.0,3100.0,6059.0,...,6.059063e+13,0.025595,2017-01-01,False,False,False,True,False,False,False
1,1387261,17052889,NaN,1.0,2.0,NaN,1.0,1465.0,1465.0,6111.0,...,6.111001e+13,0.055619,2017-01-01,False,True,False,False,False,False,False
2,11677,14186244,NaN,2.0,3.0,NaN,2.0,1243.0,1243.0,6059.0,...,6.059022e+13,0.005383,2017-01-01,False,False,False,True,True,False,False
3,2288172,12177905,NaN,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,...,6.037300e+13,-0.103410,2017-01-01,False,False,False,False,True,False,False
4,1970746,10887214,1.0,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,...,6.037124e+13,0.006940,2017-01-01,False,False,False,False,True,False,False


**Dataframe Cleaning**
- drop redundant columns
- drop columns with no variance (single values columns)
- 